#  Cadences in a Corpus



## A. Import Intervals and Other Code

* See the Corpus Methods notebook for details on the various options for local and remote files


In [2]:
import intervals
from intervals import * 
from intervals import main_objs
import intervals.visualizations as viz
import pandas as pd
import re
import altair as alt 
from ipywidgets import interact
from pandas.io.json import json_normalize
from pyvis.network import Network
from IPython.display import display
import requests
import os
import glob as glob

MYDIR = ("saved_csv")
CHECK_FOLDER = os.path.isdir(MYDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MYDIR)
    print("created folder : ", MYDIR)
else:
    print(MYDIR, "folder already exists.")
    
MUSDIR = ("Music_Files")
CHECK_FOLDER = os.path.isdir(MUSDIR)

# If folder doesn't exist, then create it.
if not CHECK_FOLDER:
    os.makedirs(MUSDIR)
    print("created folder : ", MUSDIR)
else:
    print(MUSDIR, "folder already exists.")

saved_csv folder already exists.
Music_Files folder already exists.


## B. Importing Corpus

One method:

* The pieces are provided as a **list**, within square brackets and separated by commas.  
* Complete URLs of each piece in this case
* The bracketed list is then contained within the parentheses of `CorpusBase()`
* For example: 

>`corpus CorpusBase(['https://crimproject.org/mei/CRIM_Mass_0006_1.mei', 'https://crimproject.org/mei/CRIM_Mass_0006_2.mei', 'https://crimproject.org/mei/CRIM_Mass_0006_3.mei'])`
    
Another method:

* Simply load **all** the pieces in the local `Music_Files/` folder:

> `piece_list = []
for name in glob.glob('Music_Files/*'):
    piece_list.append(name)
piece_list`

Read the documentation:  `print(CorpusBase.batch.__doc__)`


### Full CRIM Corpus

* Here we omit various monophonic pieces and a few others for which there are errors.

>`piece_list = []`<br>
    
> `raw_prefix = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/"`<br>
    
> `URL = "https://api.github.com/repos/CRIM-Project/CRIM-online/git/trees/990f5eb3ff1e9623711514d6609da4076257816c"`<br>
> `piece_json = requests.get(URL).json()`

* A list of files to exclude:

> `exclude_list = ['CRIM_Model_0003.mei', 'CRIM_Model_0004.mei', 'CRIM_Model_0005.mei', 'CRIM_Model_0006.mei', 'CRIM_Model_0007.mei','CRIM_Model_0022.mei', 'CRIM_Model_0028.mei', 'CRIM_Model_0035.mei', 'CRIM_Mass_0029_4.mei', 'CRIM_Mass_0049_2.mei','CRIM_Mass_0049_5.mei']`

*  The following ensures that we don't try to analyze the 'Mass head only' files, which have no musical content:

>`pattern = 'CRIM_Mass_([0-9]{4}).mei'`

* Now the request for all the files

>`for p in piece_json["tree"]:
p_name = p["path"]
if re.search(pattern, p_name):
    pass
elif p_name in exclude_list:
    pass
else:
    piece_list.append(raw_prefix + p["path"])`

In [14]:
# this will pull ALL pieces from CRIM on Github
# Note that we exclude various monophonic pieces (which have no contrapuntal cadences)
# and also a few pieces that seem to throw errors for reasons we don't understand.
corpus_list = []
raw_prefix = "https://raw.githubusercontent.com/CRIM-Project/CRIM-online/master/crim/static/mei/MEI_4.0/"
URL = "https://api.github.com/repos/CRIM-Project/CRIM-online/git/trees/990f5eb3ff1e9623711514d6609da4076257816c"
piece_json = requests.get(URL).json()

# list of files to exclude
exclude_list = ['CRIM_Model_0003.mei', 'CRIM_Model_0004.mei', 'CRIM_Model_0005.mei', 'CRIM_Model_0006.mei', 
             'CRIM_Model_0007.mei',
            'CRIM_Model_0022.mei', 'CRIM_Model_0028.mei', 'CRIM_Model_0035.mei', 'CRIM_Mass_0029_4.mei', 
             'CRIM_Mass_0049_2.mei',
            'CRIM_Mass_0049_5.mei']

# this ensures that we don't try to analyze the 'Mass head only' files, which have no musical content

pattern = 'CRIM_Mass_([0-9]{4}).mei'

# and now the request for all the files
for p in piece_json["tree"]:
    p_name = p["path"]
    if re.search(pattern, p_name):
        pass
    elif p_name in exclude_list:
        pass
    else:
        corpus_list.append(raw_prefix + p["path"])

In [4]:
# use this to make a list of all the pieces in the Music_Files folder

corpus_list = []
for name in glob.glob('Music_Files/*'):
    corpus_list.append(name)
#     print(name)
corpus = CorpusBase(corpus_list)

Successfully imported Music_Files/Morley_1597_068.xml
Successfully imported Music_Files/Morley_1597_135_2.xml
Successfully imported Music_Files/Morley_1597_132_08.xml
Successfully imported Music_Files/Morley_1597_131_6.xml
Successfully imported Music_Files/Morley_1597_131_7.xml
Successfully imported Music_Files/Morley_1597_132_09.xml
Successfully imported Music_Files/Morley_1597_135_3.xml
Successfully imported Music_Files/Morley_1597_072_4.xml
Successfully imported Music_Files/Morley_1597_135_1.xml
Successfully imported Music_Files/Morley_1597_072_6.xml
Successfully imported Music_Files/Morley_1597_131_5.xml
Successfully imported Music_Files/Morley_1597_076_3.xml
Successfully imported Music_Files/Morley_1597_072_3.xml
Successfully imported Music_Files/Morley_1597_131_1.xml
Successfully imported Music_Files/Morley_1597_072_2.xml
Successfully imported Music_Files/Morley_1597_135_5.xml
Successfully imported Music_Files/Morley_1597_131_3.xml
Successfully imported Music_Files/Morley_1597_07

In [22]:
# Run this to check for files that fail to process

# corpus.batch(func=ImportedPiece.cadences, verbose=True)    

### C. 1 Find the Cadences in the Corpus

* Sample code (remember to omit "()" after the cadences function!

>`func = ImportedPiece.cadences
list_of_dfs = corpus.batch(func=func, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=False)`

* Suggested reorganization of columns in the output:

>`col_list = ['Composer', 'Title', 'Measure', 'Beat', 'CadType', 'Tone','Evaded', 'LeadingTones', 'Low','RelLow','RelTone','Progress','SinceLast','ToNext', 'Validation', 'Comments']`
    
>`combined_df = combined_df[col_list]`

In [17]:
func = ImportedPiece.cadences
list_of_dfs = corpus.batch(func=func, metadata=True)
combined_df = pd.concat(list_of_dfs, ignore_index=False)


combined_df['Validation'] = ""
combined_df['Comments'] = ""
col_list = ['Composer', 'Title', 'Measure', 'Beat', 'CadType', 'Tone','CVFs',
                'LeadingTones', 'Sounding', 'Low','RelLow','RelTone',
                'Progress','SinceLast','ToNext', 'Validation', 'Comments']
combined_df = combined_df[col_list]
# combined_df.to_csv("full_corpus_results.csv")
combined_df

,Composer,Title,Measure,Beat,CadType,Tone,CVFs,LeadingTones,Sounding,Low,RelLow,RelTone,Progress,SinceLast,ToNext,Validation,Comments
52.0,Not found,Morley_p_068.Aria,7,3.000000,Authentic,G,TCB,1.0,3.0,G3,P1,P8,0.388060,52.0,24.0,,
76.0,Not found,Morley_p_068.Aria,10,3.000000,Authentic,G,tCB,1.0,3.0,G3,P1,P8,0.567164,24.0,20.0,,
96.0,Not found,Morley_p_068.Aria,13,1.000000,Authentic,A,TCB,1.0,3.0,A3,M2,M2,0.716418,20.0,8.0,,
104.0,Not found,Morley_p_068.Aria,14,1.000000,Clausula Vera,G,CT,1.0,3.0,G3,P1,P8,0.776119,8.0,8.0,,
112.0,Not found,Morley_p_068.Aria,15,1.000000,Clausula Vera,D,CT,0.0,3.0,D4,P5,P5,0.835821,8.0,4.0,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8.0,Morley,Plaine_1597_132_7,1,2.333333,NaN,A,CCts,NaN,4.0,F3,P1,M3,1.000000,8.0,4.0,,
12.0,Morley,Plaine_1597_132_6,2,3.000000,Authentic,F,CtB,1.0,4.0,F3,P1,P8,1.000000,12.0,4.0,,
12.0,Morley,Plaine_1597_132_4,2,3.000000,Phrygian Clausula Vera,E,CT,1.0,4.0,E3,P1,P8,0.857143,12.0,4.0,,
12.0,Morley,Plaine_1597_132_10,2,3.000000,Authentic,G,CTB,1.0,4.0,G2,P1,P8,1.000000,12.0,4.0,,


### C.2.  Summary by Type, Tone, etc

* Here you can report an inventory of cadences by **type** and **tone** (and **evaded** status):
> `combined_df['Tone'].value_counts().to_frame()`




In [12]:
combined_df['CadType'].value_counts().to_frame()


,CadType
Authentic,32
Clausula Vera,26
Evaded Clausula Vera,13
Phrygian Clausula Vera,5
Phrygian,1
Evaded Altizans Only,1
Abandoned Authentic,1
Evaded Authentic,1


* Or, various groupings:


>`combined_df.groupby(['CadType', 'Tone', 'Evaded']).size().reset_index(name='counts')`

In [27]:

grouped_types = combined_df.groupby(['Sounding', 'CVFs', 'Tone', 'CadType']).size().reset_index(name='counts')
grouped_types

,Sounding,CVFs,Tone,CadType,counts
0,1.0,Cx,B,Abandoned Authentic,1
1,2.0,BC,D,Authentic,1
2,2.0,CB,B,Authentic,3
3,2.0,CB,D,Authentic,1
4,2.0,CT,A,Clausula Vera,1
5,2.0,CT,D,Clausula Vera,3
6,2.0,CT,F#,Phrygian Clausula Vera,2
7,2.0,CT,G,Clausula Vera,4
8,2.0,Cu,B,Evaded Authentic,1
9,2.0,TC,A,Clausula Vera,1


In [31]:
grouped_types.to_csv("Morley_Cadences_Grouped_9_29.csv")

In [ ]:
func = ImportedPiece.verovioCadences
corpus.batch(func=func)

In [30]:
piece_list = []
for name in glob.glob('Music_Files/*'):
    piece = importScore(name)
    piece.verovioCadences()

Previously imported piece detected.
Results:
File Name:  Morley_1597_068
Not found
Morley_p_068.Aria
Cadence End Measure: 7
Beat:  3.0
Cadence Tone:  G
Cadence Type:  Authentic
Cadential Voice Functions:  TCB


Results:
File Name:  Morley_1597_068
Not found
Morley_p_068.Aria
Cadence End Measure: 10
Beat:  3.0
Cadence Tone:  G
Cadence Type:  Authentic
Cadential Voice Functions:  tCB


Results:
File Name:  Morley_1597_068
Not found
Morley_p_068.Aria
Cadence End Measure: 13
Beat:  1.0
Cadence Tone:  A
Cadence Type:  Authentic
Cadential Voice Functions:  TCB


Results:
File Name:  Morley_1597_068
Not found
Morley_p_068.Aria
Cadence End Measure: 14
Beat:  1.0
Cadence Tone:  G
Cadence Type:  Clausula Vera
Cadential Voice Functions:  CT


Results:
File Name:  Morley_1597_068
Not found
Morley_p_068.Aria
Cadence End Measure: 15
Beat:  1.0
Cadence Tone:  D
Cadence Type:  Clausula Vera
Cadential Voice Functions:  CT


...

[Warning] Measure range start for selection '0-1' could not be found.


Previously imported piece detected.
Previously imported piece detected.
Previously imported piece detected.
Results:
File Name:  Morley_1597_132_06
Morley
Plaine_1597_132_6
Cadence End Measure: 2
Beat:  3.0
Cadence Tone:  F
Cadence Type:  Authentic
Cadential Voice Functions:  CtB


Previously imported piece detected.
Results:
File Name:  Morley_1597_132_04
Morley
Plaine_1597_132_4
Cadence End Measure: 2
Beat:  3.0
Cadence Tone:  E
Cadence Type:  Phrygian Clausula Vera
Cadential Voice Functions:  CT


Previously imported piece detected.
Results:
File Name:  Morley_1597_132_10
Morley
Plaine_1597_132_10
Cadence End Measure: 2
Beat:  3.0
Cadence Tone:  G
Cadence Type:  Authentic
Cadential Voice Functions:  CTB


Previously imported piece detected.
Results:
File Name:  Morley_1597_132_05
Morley
Plaine_1597_132_5
Cadence End Measure: 2
Beat:  3.0
Cadence Tone:  A
Cadence Type:  Authentic
Cadential Voice Functions:  tCB


In [ ]:
corpus = CorpusBase(piece_list)
func = ImportedPiece.cadences
corpus.batch(func=func)